In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
from otc.metrics.metrics import effective_spread

import wandb

from tqdm.auto import tqdm

from scipy.stats import wilcoxon
import warnings


/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set here globally
exchange = "cboe"
models = ["gbm", "classical"]
subset = "test"  # "all"
strategy = "transfer"  # "supervised"


In [3]:
key = f"{exchange}_all_{strategy}_{subset}"
dataset = f"fbv/thesis/{exchange}_{strategy}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(dataset)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in models:
    results = f"fbv/thesis/{exchange}_{model}_{strategy}_{subset}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact cboe_transfer_raw:latest, 663.29MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_gbm_transfer_test:latest, 67.57MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_classical_transfer_test:latest, 365.80MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [4]:
# p. 35-38
columns = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if subset == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=columns
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=columns
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif subset == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )


results = []
for i, model in tqdm(enumerate(models)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=models)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


2it [00:02,  1.28s/it]


In [5]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        ("gbm", "gbm(classical-size)"),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex)",
        ),
    ]
]


In [6]:
results_data


gbm classical                                    \
         gbm(classical-size)  tick(ex) quote(ex) lr(ex) emo(ex) clnv(ex)   
index                                                                      
24700819                  -1       1.0      -1.0   -1.0    -1.0     -1.0   
24700261                  -1       1.0      -1.0   -1.0    -1.0     -1.0   
24700842                  -1       1.0      -1.0    1.0     1.0      1.0   
24695039                   1      -1.0      -1.0   -1.0    -1.0     -1.0   
24700841                   1       1.0       1.0    1.0     1.0      1.0   
...                      ...       ...       ...    ...     ...      ...   
37155409                   1       1.0      -1.0   -1.0    -1.0     -1.0   
37155410                   1       1.0       1.0    1.0     1.0      1.0   
37104659                  -1      -1.0       1.0    1.0     1.0      1.0   
37108365                   1      -1.0      -1.0   -1.0    -1.0     -1.0   
37155411                   1      -1.0      -1.0   -1.0    -1.0     -1.0   

                                                                                       
         trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex)  
index                                                                                  
24700819                                               -1.0                            
24700261                                               -1.0                            
24700842                                               -1.0                            
24695039                                                1.0                            
24700841                                                1.0                            
...                                                     ...                            
37155409                                               -1.0                            
37155410                                                1.0                            
37104659                                               -1.0                            
37108365                                                1.0                            
37155411                                                1.0                            

[12791228 rows x 7 columns]

### Robustness Checks

In [7]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "ttm <= 1 month",
    "ttm (1-2] month",
    "ttm (2-3] month",
    "ttm (3-6] month",
    "ttm (6-12] month",
    "ttm > 12 month",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock options",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "myn <= 0.7",
    "myn (0.7-0.9]",
    "myn (0.9-1.1]",
    "myn (1.1-1.3]",
    "myn > 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)
half_spread = np.where(ask >= bid, (ask - bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        ((mid - half_spread) < trade_price) & (trade_price < (mid + half_spread)),
        1,  # inside
        np.where(
            (trade_price == (mid - half_spread)) | ((mid + half_spread) == trade_price),
            2,  # at quotes
            np.where(
                (trade_price < (mid - half_spread))
                | ((mid + half_spread) > trade_price),
                3,
                4,
            ),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "ask_ex",
        "bid_ex",
        "year",
    ],
    inplace=True,
)


In [8]:
X_print.head(20)


,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid
index,,,,,,,,,,
24700819,-1,1.730000,C,Others,"(0,1]",2015,ttm (1-2] month,myn (0.9-1.1],unknown,NaN
24700261,-1,0.600000,C,Stock options,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],at quotes,0.80
24700842,-1,1.300000,C,Others,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],unknown,NaN
24695039,1,1.280000,C,Others,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],unknown,NaN
24700841,-1,1.040000,C,Others,>11,2015,ttm <= 1 month,myn (0.9-1.1],at quotes,1.00
182536,-1,3.300000,P,Others,"(0,1]",2015,ttm (3-6] month,myn (0.7-0.9],unknown,NaN
24700164,1,0.250000,C,Stock options,>11,2015,ttm <= 1 month,myn (0.9-1.1],unknown,NaN
24323781,1,64.239998,P,Index option,"(0,1]",2015,ttm (3-6] month,myn (0.9-1.1],unknown,NaN
24700851,1,8.950000,C,Stock options,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],inside,8.50


In [9]:
X_print = pd.concat([X_print, results_data], axis=1)


In [10]:
X_print.head()


,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,"(gbm, gbm(classical-size))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex))"
index,,,,,,,,,,,,,,,,,
24700819,-1,1.73,C,Others,"(0,1]",2015,ttm (1-2] month,myn (0.9-1.1],unknown,NaN,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
24700261,-1,0.60,C,Stock options,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],at quotes,0.8,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
24700842,-1,1.30,C,Others,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],unknown,NaN,-1,1.0,-1.0,1.0,1.0,1.0,-1.0
24695039,1,1.28,C,Others,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],unknown,NaN,1,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
24700841,-1,1.04,C,Others,>11,2015,ttm <= 1 month,myn (0.9-1.1],at quotes,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0


## Results Set Generation

In [74]:
LUT = {
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    # "(": "(",  # put interval start in math env
    # "]": "]",  # put interval end in math env
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Location to Quote",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [75]:
def set_tex_style(styler, caption, label):
    res = (
        styler.set_caption(caption)
        # .hide(axis="index")
        .apply(
            highlight_max, props="font-weight:bold;", axis=0
        )  # optionaly set subset....
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        # .hide(axis="index", level=0)
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [76]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


In [77]:
results_data.columns.tolist()


[('gbm', 'gbm(classical-size)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex)')]

## Accuracy Calculation

In [78]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        acc_tot = accuracy_score(
            X_print["buy_sell"].astype("int8"), X_print[classifier]
        )

        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
        res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


100%|██████████| 7/7 [02:47<00:00, 23.92s/it]


In [79]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [80]:
master


gbm  classical             \
                                   gbm(classical-size)   tick(ex)  quote(ex)   
Option Type       C                          71.726305  48.639452  61.855015   
                  P                          72.263779  49.446618  62.281154   
                  all                        71.970823  49.006663  62.048882   
Security Type     Index option               67.418516  48.174157  53.865554   
                  Others                     74.140903  48.745063  66.064760   
                  Stock options              71.521690  49.197934  61.193841   
                  all                        71.970823  49.006663  62.048882   
Trade Size        (0,1]                      70.805402  48.806380  58.840355   
                  (1,3]                      71.384306  48.943375  62.018777   
                  (3,5]                      71.967362  48.731932  62.431819   
                  (5,11]                     71.571455  48.827326  62.782615   
                  >11                        74.498617  49.696146  65.364517   
                  all                        71.970823  49.006663  62.048882   
Year              2015                       71.497521  48.779877  63.920984   
                  2016                       71.624954  48.904024  62.841641   
                  2017                       72.398824  49.144306  60.970926   
                  all                        71.970823  49.006663  62.048882   
Time to Maturity  ttm <= 1 month             72.015004  48.975387  62.861248   
                  ttm (1-2] month            72.648398  49.791415  62.824691   
                  ttm (2-3] month            72.321337  48.990408  61.869454   
                  ttm (3-6] month            72.895032  48.819425  61.102543   
                  ttm (6-12] month           71.950704  48.591253  60.698895   
                  ttm > 12 month             66.808042  48.433594  54.322709   
                  all                        71.970823  49.006663  62.048882   
Moneyness         myn <= 0.7                 73.013351  49.728784  56.297593   
                  myn (0.7-0.9]              72.201759  48.423386  60.706421   
                  myn (0.9-1.1]              72.631488  49.278613  63.650009   
                  myn (1.1-1.3]              66.175326  47.879624  55.822384   
                  myn > 1.3                  64.543886  47.501014  53.703573   
                  all                        71.970823  49.006663  62.048882   
Location to Quote at mid                     67.960073  45.333930  49.906025   
                  inside                     72.055702  49.511694  67.394807   
                  at quotes                  74.916997  49.247537  38.361322   
                  outside                    73.276470  45.884669  42.765060   
                  unknown                    75.364515  49.501980  38.737550   
                  all                        71.970823  49.006663  62.048882   

                                                                     \
                                       lr(ex)    emo(ex)   clnv(ex)   
Option Type       C                 61.529399  48.509719  53.209126   
                  P                 62.011341  48.803783  53.296756   
                  all               61.748653  48.643500  53.248992   
Security Type     Index option      53.648287  42.012213  43.576012   
                  Others            65.669597  50.246585  55.151702   
                  Stock options     60.924802  48.636117  53.419253   
                  all               61.748653  48.643500  53.248992   
Trade Size        (0,1]             58.626470  46.875427  50.730336   
                  (1,3]             61.732442  48.552952  52.889873   
                  (3,5]             62.139861  48.989046  53.488609   
                  (5,11]            62.449727  48.701236  53.621842   
                  >11               64.960174  50.774968  56.436478   
                

In [81]:
master.style.pipe(
    set_tex_style, caption=("master-short", "master-long"), label=f"{key}-master"
)


## Effective Spread 🚧

In [82]:
classifiers


[('gbm', 'gbm(classical-size)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex)')]

In [83]:
eff_dfs = []


def stats(x, classifier):

    nom = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="nominal")
    rel = (
        effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="relative")
        * 100
    )

    # eff_spread_pred = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="none")
    # eff_spread_true = effective_spread(x["buy_sell"], x["TRADE_PRICE"], x["mid"], mode="none")
    # wilcoxon_res  = wilcoxon(eff_spread_pred, eff_spread_true, nan_policy="omit", zero_method="zsplit")

    return pd.Series(
        {
            "nominal": nom,
            "rel": rel,
            # 'statistic':wilcoxon_res.statistic,
            # 'pvalue':wilcoxon_res.pvalue
        }
    )


for criterion in tqdm(criterions):
    results = []

    for classifier in tqdm(classifiers):
        res = X_print.groupby([criterion])[
            ["TRADE_PRICE", "mid", classifier, "buy_sell"]
        ].apply(stats, classifier)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1, keys=classifiers).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}-eff-spread",
    )

    # store all result sets for later use
    eff_dfs.append(result_df)


100%|██████████| 7/7 [00:47<00:00,  6.86s/it]


In [84]:
master = pd.concat(eff_dfs, axis=1, keys=LUT_INDEX.values()).T


In [85]:
master


gbm                 \
                                   gbm(classical-size)                  
                                               nominal            rel   
Option Type       C                           0.043221  9.682117e+302   
                  P                           0.036723  1.159862e+303   
Security Type     Index option                0.094955  -3.294041e+01   
                  Others                      0.027861   7.634214e+00   
                  Stock options               0.039984   7.224868e+00   
Trade Size        (0,1]                       0.032503  6.670910e+302   
                  (1,3]                       0.040007   7.219027e-01   
                  (3,5]                       0.039417   6.362226e+00   
                  (5,11]                      0.038551   7.967805e+00   
                  >11                         0.052950  8.794507e+302   
Year              2015                        0.061394  6.268773e+303   
                  2016                        0.044715   6.958982e+00   
                  2017                        0.032913   7.287113e+00   
Time to Maturity  ttm <= 1 month              0.032689  6.035006e+302   
                  ttm (1-2] month             0.038471   6.346819e+00   
                  ttm (2-3] month             0.036274   3.748553e+00   
                  ttm (3-6] month             0.045517  1.514673e+303   
                  ttm (6-12] month            0.061025  2.620955e+303   
                  ttm > 12 month              0.103297  3.828225e+303   
Moneyness         myn <= 0.7                  0.032480   1.678756e+01   
                  myn (0.7-0.9]               0.029111   1.229772e+01   
                  myn (0.9-1.1]               0.038416   5.878038e+00   
                  myn (1.1-1.3]               0.075553  2.848268e+303   
                  myn > 1.3                   0.102083   1.044778e+00   
Location to Quote at mid                      0.000000   0.000000e+00   
                  inside                      0.057293   7.363938e+00   
                  at quotes                  -0.037148   1.441746e+01   
                  outside                    -0.046858  -1.113457e+00   
                  unknown                     0.031339            inf   

                                   classical                           \
                                    tick(ex)                quote(ex)   
                                     nominal            rel   nominal   
Option Type       C                 0.013384  9.682117e+302  0.131108   
                  P                 0.014528  1.159862e+303  0.118537   
Security Type     Index option      0.034780  -3.593394e+01  0.268497   
                  Others            0.010580   1.020192e+00  0.078642   
                  Stock options     0.013210   2.864617e+00  0.130577   
Trade Size        (0,1]             0.017141  6.670910e+302  0.147487   
                  (1,3]             0.014669  -3.426757e+00  0.127715   
                  (3,5]             0.014451   2.328012e+00  0.118979   
                  (5,11]            0.010608   2.243740e+00  0.113479   
                  >11               0.011499  8.794507e+302  0.109310   
Year              2015              0.025516  6.268773e+303  0.140507   
                  2016              0.012870   2.520000e+00  0.122155   
                  2017              0.013582   1.924026e+00  0.126984   
Time to Maturity  ttm <= 1 month    0.013230  6.035006e+302  0.107739   
                  ttm (1-2] month   0.010397   2.592617e+00  0.103392   
                  ttm (2-3] month   0.006046   1.261972e+00  0.130511   
                  ttm (3-6] month   0.015580  1.514673e+303  0.137988   
                  ttm (6-12] month  0.018053  2.620955e+303  0.169784   
                  ttm > 12 month    0.032801  3.828225e+303  0.303339   
Moneyness         myn <= 0.7        0.012614   9.740130e+00  0.127572   
                  myn (0.7-0.9]    

In [86]:
master.iloc[:, 0:20]


gbm                 \
                                   gbm(classical-size)                  
                                               nominal            rel   
Option Type       C                           0.043221  9.682117e+302   
                  P                           0.036723  1.159862e+303   
Security Type     Index option                0.094955  -3.294041e+01   
                  Others                      0.027861   7.634214e+00   
                  Stock options               0.039984   7.224868e+00   
Trade Size        (0,1]                       0.032503  6.670910e+302   
                  (1,3]                       0.040007   7.219027e-01   
                  (3,5]                       0.039417   6.362226e+00   
                  (5,11]                      0.038551   7.967805e+00   
                  >11                         0.052950  8.794507e+302   
Year              2015                        0.061394  6.268773e+303   
                  2016                        0.044715   6.958982e+00   
                  2017                        0.032913   7.287113e+00   
Time to Maturity  ttm <= 1 month              0.032689  6.035006e+302   
                  ttm (1-2] month             0.038471   6.346819e+00   
                  ttm (2-3] month             0.036274   3.748553e+00   
                  ttm (3-6] month             0.045517  1.514673e+303   
                  ttm (6-12] month            0.061025  2.620955e+303   
                  ttm > 12 month              0.103297  3.828225e+303   
Moneyness         myn <= 0.7                  0.032480   1.678756e+01   
                  myn (0.7-0.9]               0.029111   1.229772e+01   
                  myn (0.9-1.1]               0.038416   5.878038e+00   
                  myn (1.1-1.3]               0.075553  2.848268e+303   
                  myn > 1.3                   0.102083   1.044778e+00   
Location to Quote at mid                      0.000000   0.000000e+00   
                  inside                      0.057293   7.363938e+00   
                  at quotes                  -0.037148   1.441746e+01   
                  outside                    -0.046858  -1.113457e+00   
                  unknown                     0.031339            inf   

                                   classical                           \
                                    tick(ex)                quote(ex)   
                                     nominal            rel   nominal   
Option Type       C                 0.013384  9.682117e+302  0.131108   
                  P                 0.014528  1.159862e+303  0.118537   
Security Type     Index option      0.034780  -3.593394e+01  0.268497   
                  Others            0.010580   1.020192e+00  0.078642   
                  Stock options     0.013210   2.864617e+00  0.130577   
Trade Size        (0,1]             0.017141  6.670910e+302  0.147487   
                  (1,3]             0.014669  -3.426757e+00  0.127715   
                  (3,5]             0.014451   2.328012e+00  0.118979   
                  (5,11]            0.010608   2.243740e+00  0.113479   
                  >11               0.011499  8.794507e+302  0.109310   
Year              2015              0.025516  6.268773e+303  0.140507   
                  2016              0.012870   2.520000e+00  0.122155   
                  2017              0.013582   1.924026e+00  0.126984   
Time to Maturity  ttm <= 1 month    0.013230  6.035006e+302  0.107739   
                  ttm (1-2] month   0.010397   2.592617e+00  0.103392   
                  ttm (2-3] month   0.006046   1.261972e+00  0.130511   
                  ttm (3-6] month   0.015580  1.514673e+303  0.137988   
                  ttm (6-12] month  0.018053  2.620955e+303  0.169784   
                  ttm > 12 month    0.032801  3.828225e+303  0.303339   
Moneyness         myn <= 0.7        0.012614   9.740130e+00  0.127572   
                  myn (0.7-0.9]    

In [87]:
master.style.pipe(
    set_tex_style,
    caption=("master-short", "master-long"),
    label=f"{key}-master-eff-spread",
)


## Change in Parenthesis 🅾️

```latex
# https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290
\begin{table}
    \centering
    \caption{test of combination with change}
    \label{tab:combo}
    \begin{tabular}{lSSSSSSSS}
        \toprule
        {} & \multicolumn{2}{l}{Index option} & \multicolumn{2}{l}{Others} & \multicolumn{2}{l}{Stock options} & \multicolumn{2}{l}{all} \\
        \midrule
        classical-size & 1.0 & \parl-56.42\parr & 2.0 & \parl-74.35\parr & -73.5 & \parl-143.93\parr & 5.0 & \parl-67.33\parr \\
        \bottomrule
        \end{tabular}
\end{table}
```

In [25]:
foo = pd.DataFrame([[4, 3, 8, 5]])
bar = pd.DataFrame([[1.1, 2, 73, 5]], columns=foo.columns, index=foo.index)


In [26]:
def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    for i, mul_col in enumerate(combo.columns):

        # define custom brackets that are not parsed by sunitx
        combo[[(mul_col[0], "pm")]] = (
            "\parl" + (combo[mul_col] - base[mul_col[0]]).round(2).astype(str) + "\parr"
        )
        # sort to group together columns
        combo.sort_index(axis=1, inplace=True)
    return combo


In [27]:
combo = combine_results(bar, foo)

# manually replace S with S[table-format=1.4(5)] if needed
combo.style.to_latex(
    f"combo.tex",
    siunitx=True,
    position_float="centering",
    hrules=True,
    clines="skip-last;data",
    label="tab:combo",
    caption="test of combination with change",
    multicol_align="l",
)


In [28]:
combo


0                    1                     2                    3  \
   nom              pm  nom              pm   nom              pm  nom   
0  1.1  \parl-2.9\parr  2.0  \parl-1.0\parr  73.0  \parl65.0\parr  5.0   

                  
              pm  
0  \parl0.0\parr